In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import nemo
import attrdict
from ruamel import yaml
from nemo.collections import tts as nemo_tts

In [11]:
MODEL_CONFIG = '/home/stanislavv/src/tts/NeMo/examples/tts/configs/fasterspeech.yaml'
yaml_loader = yaml.YAML(typ="safe")
with open(MODEL_CONFIG) as f:
    pp_config = attrdict.AttrDict(yaml_loader.load(f))
pp_config.sample_rate = 22025
pp_config.AudioToMelSpectrogramPreprocessor

AttrDict({'sample_rate': 24000, 'window_size': 0.02, 'window_stride': 0.01, 'window': 'hann', 'normalize': None, 'n_fft': 512, 'preemph': None, 'features': 64, 'lowfreq': 0, 'highfreq': None, 'log': True, 'log_zero_guard_type': 'clamp', 'log_zero_guard_value': 1e-05, 'dither': 0.0, 'pad_to': 16, 'frame_splicing': 1, 'stft_conv': True, 'pad_value': -11.52, 'mag_power': 1.0})

In [15]:
runner = nemo.core.NeuralModuleFactory(
    placement=nemo.core.DeviceType.GPU,
)

[NeMo W 2020-04-18 19:11:23 deprecated:68] Function ``_get_trainer`` is deprecated. It is going to be removed in the future version.


In [17]:
m = nemo_tts.TalkNet(10, 128, 0, pp_config.JasperEncoder, 128)

In [29]:
m.train();

In [31]:
m.eval();

In [32]:
m.training, m.jasper.training

(False, False)

In [2]:
import nemo
import torch
import numpy as np

from nemo.collections import tts as nemo_tts

In [3]:
B, T, C, num_classes = 32, 100, 15, 32
y_pred = torch.normal(0.0, 1.0, (B, T, C))
y_true = torch.tensor(np.random.choice(num_classes, size=(B, T), replace=True), dtype=torch.long)
(y_pred.shape, y_pred.dtype), (y_true.shape, y_true.dtype)

((torch.Size([32, 100, 15]), torch.float32),
 (torch.Size([32, 100]), torch.int64))

In [4]:
nemo_tts.parts.dmld_loss(y_pred, y_true, num_classes=num_classes)

tensor([[0.8962, 1.2270, 0.7164,  ..., 1.0317, 0.7999, 0.8104],
        [1.8184, 1.3575, 1.0692,  ..., 1.2355, 1.8862, 0.5127],
        [0.9521, 0.9255, 1.1861,  ..., 2.7445, 1.4190, 1.5864],
        ...,
        [1.1155, 0.8727, 0.8439,  ..., 1.1352, 2.7130, 1.0487],
        [0.8769, 1.7948, 2.0736,  ..., 0.9733, 0.9567, 0.5358],
        [2.1465, 1.7333, 0.6418,  ..., 1.3870, 1.1792, 2.1006]])

In [11]:
nemo_tts.parts.dmld_sample(y_pred, num_classes=num_classes)

tensor([[18.8765,  0.0000,  0.0000,  ...,  0.0000, 31.0000, 30.9614],
        [ 0.0000,  0.0000, 15.8390,  ..., 16.2855, 25.4399,  0.0000],
        [16.8917,  0.0000,  0.0000,  ..., 28.1443,  5.3356, 31.0000],
        ...,
        [31.0000, 31.0000, 13.4109,  ..., 20.2352, 31.0000, 31.0000],
        [ 0.0000,  1.4237, 31.0000,  ..., 31.0000,  0.0000, 31.0000],
        [26.8359, 19.5081,  0.0000,  ..., 23.4756, 18.5434, 31.0000]])

In [76]:
num_classes = 32
classes = np.arange(num_classes).tolist()
print(*classes)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31


In [89]:
num_classes = 32
classes = np.arange(num_classes).tolist()

k, c = 1, num_classes - 1
while c < 500:
    k *= 1.5
    c += k
    classes.append(int(c))

print(len(classes))
print(*classes)

45
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 34 38 43 50 62 79 104 143 200 287 417 611


In [20]:
classes = torch.tensor(classes)
classes

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  33,  37,  45,  61,  93, 157, 285, 541])

In [23]:
dur_true = torch.randint(0, 300, (32, 500))
dur_true

tensor([[251,  18,  84,  ...,  38, 149, 150],
        [209,  56, 248,  ..., 249, 262,  56],
        [237, 290, 161,  ..., 161, 206,  68],
        ...,
        [273,  47, 247,  ..., 110, 202, 130],
        [207,  60, 171,  ..., 266, 213, 119],
        [ 82, 112,  82,  ...,  94,  93, 255]])

In [30]:
dur_true.shape

torch.Size([32, 500])

In [32]:
a = dur_true.unsqueeze(-1).repeat(1, 1, *classes.shape)
b = classes.unsqueeze(0).unsqueeze(0).repeat(*dur_true.shape, 1)
a.shape, b.shape

(torch.Size([32, 500, 40]), torch.Size([32, 500, 40]))

In [56]:
b.shape

torch.Size([32, 500, 40])

In [57]:
ii.shape

torch.Size([32, 500])

In [39]:
ii = (a - b).abs().argmin(-1)
ii

tensor([[38, 18, 36,  ..., 33, 37, 37],
        [37, 35, 38,  ..., 38, 38, 35],
        [38, 38, 37,  ..., 37, 37, 35],
        ...,
        [38, 34, 38,  ..., 36, 37, 37],
        [37, 35, 37,  ..., 38, 37, 36],
        [36, 36, 36,  ..., 36, 36, 38]])

In [59]:
b = classes.unsqueeze(0).unsqueeze(0).repeat(*ii.shape, 1)
b.gather(-1, ii.unsqueeze(-1)).squeeze()

tensor([[285,  18,  93,  ...,  37, 157, 157],
        [157,  61, 285,  ..., 285, 285,  61],
        [285, 285, 157,  ..., 157, 157,  61],
        ...,
        [285,  45, 285,  ...,  93, 157, 157],
        [157,  61, 157,  ..., 285, 157,  93],
        [ 93,  93,  93,  ...,  93,  93, 285]])